<a href="https://colab.research.google.com/github/wkweigel/NotebookExamples/blob/main/VinaDocking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a simple docking workflow using autodock vina.

* Use for docking ligands into areas occupied by native co-crystallized ligands.

General process:
1. Fetch a PDB entry with a bound ligand using a PDB ID.
  * For PDB entries containing two or more ligands, you will also need the specifiy the ligand ID for the area you want to dock into.
2. Use the ligand position to auto generate gridbox coords for vina to use.
3.



Install main packages using conda and pip

(This can take a few minutes to complete, click restart when prompted and continue)

---



In [1]:
#Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()

#Install pymol, openbabel and other packages
!mamba install -c conda-forge pymol-open-source openbabel openmm pdbfixer
!mamba install conda-forge::vina

#Install rdkit and other packages
!pip install rdkit tqdm MDAnalysis py3Dmol prolif ipython LigPrepper meeko



⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:16
🔁 Restarting kernel...

Looking for: ['pymol-open-source', 'openbabel', 'openmm', 'pdbfixer']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64  ⣾  
conda-forge/noarch    ⣾  [+] 0.2s
conda-forge/linux-64   2%
conda-forge/noarch     2%[+] 0.3s
conda-forge/linux-64   7%
conda-forge/noarch    23%[+] 0.4s
conda-forge/linux-64  16%
conda-forge/noarch    42%[+] 0.5s
conda-forge/linux-64  28%
conda-forge/noarch    58%[+] 0.6s
conda-forge/linux-64  32%
conda-forge/noarch    68%[+] 0.7s
conda-forge/linux-64  36%
conda-forge/noarch    87%[+] 0.8s
conda-forge/linux-64  40%
conda-forge/noarch    97%conda-forge/noarch                                
[+] 0.9s
conda-forge/linux-64  53%[+] 1.0s
conda-forge/linux-64  61%[+] 1.1s
conda-forge/linux-64  65%[+] 1.2s
conda-forge/linux-64  82%[+] 1.3s
conda-fo

In [1]:
import sys

#Clone main repository into colab
!git clone https://github.com/wkweigel/NotebookExamples.git

#Add the main directory to the path
main_dir = '/content/NotebookExamples/VinaExamples'
sys.path.insert(0,f'{main_dir}/')

#Grant permissions for lepro
!chmod u+x {main_dir}/bin/lepro_linux_x86

from VinaUtils import *

fatal: destination path 'NotebookExamples' already exists and is not an empty directory.


In [2]:
from pymol import cmd
import py3Dmol
import pandas as pd
import random
from openbabel import pybel
import prolif as plf
import MDAnalysis as mda
from MDAnalysis.coordinates import PDB
from MDAnalysis import Merge
from LigPrepper import pdbqt2sdf
from meeko import MoleculePreparation

import math, os, random, warnings
import numpy as np
from matplotlib import cm, colors
from matplotlib import pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdDistGeom
from rdkit.Geometry import Point3D

from vina import Vina

from IPython.display import IFrame
warnings.filterwarnings("ignore")

%config Completer.use_jedi = False



Specify Inputs

In [3]:
from pathlib import Path

#The PDB ID to fetch and prepare
pdbID='6QCD'

#The smiles of the molecules to dock
docking_smiles = ['c1cc(c(cc1[C@@H]2[C@H](Cc3c(cc(cc3O2)O)O)OC(=O)c4cc(c(c(c4)O)O)O)O)O', '	Cc1cc(c(cc1NS(=O)(=O)c2ccc(cc2C(=O)O)NS(=O)(=O)c3cc(cc(c3)Cl)Cl)Br)F']



#PDB_Folder=f'{main_dir}/{pdbID}'

#Create directories  for the current project if they dont exist
PDB_Folder=f'/content/{pdbID}'
Results_Folder=f'{PDB_Folder}/Results'

Path(PDB_Folder).mkdir(parents=True, exist_ok=True)
Path(Results_Folder).mkdir(parents=True, exist_ok=True)

Fetch and prepare protien from the PDB (skip if files already exist in the project folder)

In [4]:
receptor_pdb, ligand_pdb, ligand_mol2=fetchPDB(PDB_Folder, pdbID, ligID='QUE')

In [5]:
preview_receptor(receptor_pdb,ligand_pdb)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [6]:
center, size=calc_gridbox(receptor_pdb,ligand_pdb)

Center: {'center_x': -23.9060001373291, 'center_y': 26.01099967956543, 'center_z': 19.761500358581543}
Size: {'size_x': 14.220001220703125, 'size_y': 20.450000762939453, 'size_z': 14.439001083374023}


In [7]:
preview_gridbox(receptor_pdb, ligand_pdb, center, size)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [8]:
#Define the names to be used for the output pdbqt files
receptor_pdbqt=f'{PDB_Folder}/{pdbID}_clean.pdbqt'
ligand_pdbqt=f'{PDB_Folder}/{pdbID}_NativeLigand.pdbqt'

#Convert the pdb files with openbabel
!obabel -ipdb {receptor_pdb} -opdbqt -xr -O {receptor_pdbqt}
!obabel -ipdb {ligand_pdb} -opdbqt -O {ligand_pdbqt}

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is /content/6QCD/6QCD_clean.pdb)

1 molecule converted
1 molecule converted


In [9]:
#Guarentees the conformer generation for initializing a ligands 3D coordinates is successful
v = Vina(sf_name='vina')

#Set the receptor in vina
v.set_receptor(receptor_pdbqt)

#Make the gridmaps
v.compute_vina_maps(center=list(center.values()), box_size=list(size.values()))



for idx, smiles in enumerate(docking_smiles) :
    #create rdkit mol object from smiles
    mol = Chem.MolFromSmiles(smiles)

    #add polar Hydrogens to mol
    mol_H = Chem.AddHs(mol)

    #Embed the mol for use in meeko
    lig_mol=get_real_conformer(mol_H)
    print('Conformer generation successful.')
    print(f'Docking ligand {idx + 1}...')
    #prepare the mol in meeko
    prep = MoleculePreparation()
    mol_prep = prep.prepare(lig_mol)

    #write the final pdbqt string for use in vina
    pdbqt_string = prep.write_pdbqt_string()
    #pdbqt_string = PDBQTWriterLegacy.write_string(mol_prep)

    #Set the ligand in vina
    v.set_ligand_from_string(pdbqt_string)
    #v.set_ligand_from_file(pdbqt_string)

    pdbqt_output = f'{Results_Folder}/{pdbID}_{idx}_vina_out.pdbqt'
    sdf_output = f'{Results_Folder}/{pdbID}_{idx}_vina_out.sdf'

    #dock the ligand
    v.dock(exhaustiveness=8, n_poses=5)

    #Save the docked poses to file
    v.write_poses(pdbqt_output, n_poses=5, overwrite=True)
    !mk_export.py {pdbqt_output} -o {sdf_output}

1 failed conformer generations. Reattempting...
Conformer generation successful.
Docking ligand 1...
1 failed conformer generations. Reattempting...
Conformer generation successful.
Docking ligand 2...


In [ ]:
def view_vina_results(receptor_file, vina_out_file, native_ligand_file=None):
  """
  Display a 3D preview of a receptor and vina docking poses.

  Arguments
  ==========
  receptor_file (str): Required. The path to the pdb file acting as the receptor.

  vina_out_file (str): Required. The path to the pdbqt file produced by vina after docking.

  native_ligand_file (str): Optional. The path to the pdb file acting as the native ligand.
  """
  view = py3Dmol.view()
  view.removeAllModels()
  view.setViewStyle({'style':'outline','color':'black','width':0.1})

  view.addModel(open(receptor_file,'r').read(),format='pdb')
  Prot=view.getModel()
  Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
  view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})

  if native_ligand_file is not None:
    view.addModel(open(native_ligand_file,'r').read(),format='pdb')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'greenCarbon','radius':0.2}})

  name=vina_out_file[:-6]
  for pose in range(get_pose_count(vina_out_file)-1):
    pose_sdf = f"{name}_pose_{pose}.sdf"
    view.addModel(open(pose_sdf,'r').read(),format='sdf')
    ref_m = view.getModel()
    ref_m.setStyle({},{'stick':{'colorscheme':'redCarbon','radius':0.1}})

  view.zoomTo()
  view.show()

In [16]:
def pdbqt_to_sdf(pdbqt_file=None,output=None):

    results = [m for m in pybel.readfile(filename=pdbqt_file,format='pdbqt')]
    out=pybel.Outputfile(filename=output,format='sdf',overwrite=True)
    for pose in results:

        pose.data.update({'Pose':pose.data['MODEL']})
        pose.data.update({'Score':pose.data['REMARK'].split()[2]})
        del pose.data['MODEL'], pose.data['REMARK'], pose.data['TORSDO']

        out.write(pose)
    out.close()

def split_vina_output(pdbqt_file):
    """
    Split the docking conformations in an output pdbqt file into individual files.

    Arguments
    ==========
    pdbqt_file (str): Required. The path to the pdbqt file that will be split.

    """
    with open(pdbqt_file, 'r') as f:
        lines = f.readlines()
    name=pdbqt_file[:-6]
    pose_separator = "MODEL"
    current_structure = []
    structure_count = 0

    for line in lines:
        if line.startswith(pose_separator):
            if current_structure:
                # Save the current structure to a separate file
                output_pdbqt = f"{name}_pose_{structure_count}.pdbqt"
                output_sdf = f"{name}_pose_{structure_count}.sdf"
                with open(output_pdbqt, 'w') as output_file:
                    output_file.writelines(current_structure)
                #!mk_export.py {output_pdbqt} -o {output_sdf}
                pdbqt_to_sdf(output_pdbqt, output_sdf)
                current_structure = []
                structure_count += 1
        current_structure.append(line)

    # Save the last structure
    if current_structure:
        output_pdbqt = f"structure_{structure_count}.pdbqt"
        with open(output_pdbqt, 'w') as output_file:
            output_file.writelines(current_structure)



In [25]:
ligand=0
split_vina_output(f'{Results_Folder}/{pdbID}_{ligand}_vina_out.pdbqt')
vina_file = f'{Results_Folder}/{pdbID}_{ligand}_vina_out.pdbqt'

view_vina_results(receptor_pdb,vina_file,ligand_pdb)

/content/6QCD/Results/6QCD_1_vina_out


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [33]:
ligand=0
pose=2

split_vina_output(f'{Results_Folder}/{pdbID}_{ligand}_vina_out.pdbqt')
vina_file = f'{Results_Folder}/{pdbID}_{ligand}_vina_out.pdbqt'

view_vina_pose(receptor_pdb,vina_file, pose, ligand_pdb)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:

pose=0
pdbqt_file=f'{pdbID}_{pose}_vina_out.pdbqt'

#Split vina results into seperate pdbqt files
split_vina_output(f'/content/{pdbqt_file}')

#convert the specified pose to sdf
pdbqt2sdf(f'{pdbqt_file[:-6]}_pose_{pose}.pdbqt')


#fix_protein(filename=str(pdbID)+'_clean.pdb',addHs_pH=7.4,try_renumberResidues=True,output=str(pdbID)+'_clean_H_fix.pdb')

DockingResult=f'{pdbqt_file[:-6]}_pose_{pose}.sdf'


# load fixed protein


prot = mda.Universe(receptor_pdb)
protein_mol = plf.Molecule.from_mda(prot,NoImplicit=False)
protein_mol.n_residues

# load ligands
#lig_suppl = list(plf.sdf_supplier(DockingResult))

ligand_mol = plf.sdf_supplier(DockingResult)[0]

# display ligand
plf.display_residues(ligand_mol, size=(400, 200))

In [ ]:
# use default interactions
fp = plf.Fingerprint(["Hydrophobic", "HBDonor", "HBAcceptor", "PiStacking"])
# run on your poses
fp.run_from_iterable([ligand_mol], protein_mol)
fp.plot_lignetwork(ligand_mol, kind="frame", frame=0)